In [3]:



#edited by:Qingping Zheng
#2018-09-28


from math import log
import operator
def createDataSet():
    dataSet = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels
############################################################################################################
#计算给定数据集的信息熵
def calcShannonEnt(dataSet):
    m = len(dataSet)
    # numEntries
    labelCounts = {} #创建一个空的字典
    for featVec in dataSet:  #一个一个样本的遍历数据集
        currentLabel = featVec[-1]    #取第一个样本的最后一个属性
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0  #将样本的最后一个属性加入到字典中作为键
        labelCounts[currentLabel] += 1     #存储键的值
        shannonEnt = 0
        for key in labelCounts:
            prob = float(labelCounts[key])/m
            shannonEnt -= prob*log(prob, 2)
    return shannonEnt    
#############################################################################################################
##按照给定特征划分数据集
def splitDataSet(dataSet, axis, value):
    returnDataSet = []
    for featVec in dataSet:   #dataSet本质上是元素为列表的列表，此语句表示一个一个列表的遍历数据集
        if featVec[axis] == value:    #判断当前列表中序号为axis的元素的值是否为value
            tempVec = featVec[:axis]   #将当前列表中axis位元素之前的所有元素放入列表tempVec中
            tempVec.extend(featVec[axis+1:])#将当前列表中axis位元素之后的所有元素接到列表tempVec中
            returnDataSet.append(tempVec)  #将列表tempVec放入大列表returnDataSet中，一个列表一个列表的存放
    return returnDataSet

##############################################################################################################
##选择最优的数据集划分属性
def chooseBestFeatToSplit(dataSet):
    numOfFeatures = len(dataSet[0])-1   #找出数据集中第一个元素的特征维度，-1是因为要减去类别这一为维
    baseEnt = calcShannonEnt(dataSet)
    bestInfoGain = 0  
    bestFeature = -1
    for i in range(numOfFeatures):
        featList = [example[i] for example in dataSet]  #将第i个特征的所有取值找到 ###牛逼###
        uniqueVals = set(featList)                      #第i个特征的每种取值只取一次，相当与找到所有可能的子节点
        tempEnt = 0
        newEnt = 0
        for tempValue in uniqueVals:                    #遍历每个子节点
            subDataSet = splitDataSet(dataSet, i, tempValue)  #找到每个子节点
            prob = len(subDataSet)/float(len(dataSet))                    #计算该子节点的权重
            newEnt += prob*calcShannonEnt(subDataSet)      #计算所有子节点总的信息熵
        infoGain = baseEnt - newEnt
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
        #print(infoGain)
    #bestFeature
    return bestFeature

###########################################################################################################
#将叶子节点中出现次数最多的类标签作为该叶子节点的类标签
def majorityCnt(classList):
    classCount = {}
    for classLabel in classList:
        classCount[classLabel] = classCount.get(classLabel, 0) +1
        sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse = True)
    #sortedClassCount[1][1]
    return sortedClassCount[0][0]

###############################################################################################################
##创建决策树
def createTree(dataSet, labels):
    tempLabels = labels[:]
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:    #只剩下类标签无其他属性时
        return majorityCnt(classList)
    bestFeat = chooseBestFeatToSplit(dataSet)
    bestFeatLabel = tempLabels[bestFeat]  #使属性有明确的含义
    myTree = {bestFeatLabel:{}}
    del(tempLabels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueValues = set(featValues)
    for values in uniqueValues:
        subLabels = tempLabels[:]
        myTree[bestFeatLabel][values] = createTree(splitDataSet(dataSet, bestFeat, values), subLabels)
    return myTree
        
#############################################################################################################
#分类函数
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree.keys())
    secondDict = inputTree[firstStr[0]]
    featIndex = featLabels.index(firstStr[0])
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], featLabels, testVec)
            else:
                classLabel = secondDict[key]
    return classLabel
    
############################################################################################################
#存储决策树
def storeTree(inputTree, fileName):
    import pickle
    fw = open(fileName, 'wb')
    pickle.dump(inputTree, fw)
    fw.close()
    
def grabTree(fileName):
    import pickle
    fr = open(fileName, 'rb')
    return pickle.load(fr)







































    